In [1]:
!pip install pyGMM

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.5/954.5 kB 10.5 MB/s eta 0:00:00


In [2]:
!pip install xgboost

In [8]:
# Import neessary packages
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

In [4]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Direct to Data File
path = '/content/drive/My Drive/DS 340W/FinalData1.xlsx'
data = pd.read_excel(path)

In [7]:
# Split the data into training and testing sets
X = data.drop('mag', axis=1)
y = data['mag']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=892)

# The following accounts for the different data types in our code
numerical_cols = X.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

In [9]:
# Processing of numerical and categorical data in order for model to run properly
num_trans = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

cat_trans = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

pre = ColumnTransformer(
    transformers=[
        ('num', num_trans, numerical_cols),
        ('cat', cat_trans, categorical_cols)
    ])

In [10]:
# Put data into format for the model
pre.fit(X_train)

dtrain = xgb.DMatrix(pre.transform(X_train), label=y_train)
dtest = xgb.DMatrix(pre.transform(X_test), label=y_test)

In [11]:
# Specify parameters for XGBoost model
params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'logloss',
    'eta': 0.09,
    'max_depth': 3,
    'subsample': 0.6,
    'colsample_bytree': 0.8,
    'seed': 567
}

In [12]:
# Train the XGBoost model
num_round = 200
xgb_model = xgb.train(params, dtrain, num_round)

In [13]:
# Show paramter values
param_dump = xgb_model.get_dump()
#print(param_dump[0])

0:[f10<-0.769847095] yes=1,no=2,missing=2
	1:[f15<2.00001001] yes=3,no=4,missing=4
		3:[f4<-0.572518945] yes=7,no=8,missing=8
			7:leaf=0.0235055257
			8:leaf=-0.00499759987
		4:[f10<-0.883507609] yes=9,no=10,missing=10
			9:leaf=-0.012531396
			10:leaf=0.07977698
	2:[f16<2.00001001] yes=5,no=6,missing=6
		5:[f1<0.35083732] yes=11,no=12,missing=12
			11:leaf=0.0876626968
			12:leaf=0.0632510483
		6:[f19<2.00001001] yes=13,no=14,missing=14
			13:leaf=0.186818123
			14:leaf=-0.00680134585



In [14]:
# Assign param values based on param_dump above
# Values in tree may be different, we used a different seed when we assigned these values
mag = 0.806045592
vs30 = 0.0539587326
dist_rup = -0.00742871454

In [15]:
# Run function definition for GMM
# Also incorporate parm value from above

"""Idriss (2014, :cite:`idriss14`) model."""
import numpy as np

from pygmm import model

__author__ = "Albert Kottke"


class Idriss2014(model.GroundMotionModel):
    """Idriss (2014, :cite:`idriss14`) model.

    This model was developed for active tectonic regions as part of the
    NGA-West2 effort.

    Parameters
    ----------
    scenario : :class:`pygmm.model.Scenario`
        earthquake scenario

    """

    NAME = "Idriss (2014)"
    ABBREV = "I14"

    # Reference velocity (m/s)
    V_REF = 1200.0

    # Load the coefficients for the model
    COEFF = dict(
        small=model.load_data_file("idriss_2014-small.csv", 2),
        large=model.load_data_file("idriss_2014-large.csv", 2),
    )
    PERIODS = COEFF["small"]["period"]

    INDEX_PGA = 0
    INDICES_PSA = np.arange(22)

    PARAMS = [
        model.NumericParameter("dist_rup", True, None, dist_rup),
        model.NumericParameter("mag", True, mag, None),
        model.NumericParameter("v_s30", True, vs30, 1200),
        model.CategoricalParameter("mechanism", True, ["SS", "RS"], "SS"),
    ]


    def __init__(self, scenario: model.Scenario):
        """Initialize the model."""
        super().__init__(scenario)
        self._ln_resp = self._calc_ln_resp()
        self._ln_std = self._calc_ln_std()



    def _calc_ln_resp(self) -> np.ndarray:
        s = self._scenario
        c = self.COEFF["small"] if s.mag <= 6.75 else self.COEFF["large"]

        if s.mechanism == "RS":
            flag_mech = 1
        else:
            # SS/RS/U
            flag_mech = 0

        f_mag = c.alpha_1 + c.alpha_2 * s.mag + c.alpha_3 * (8.5 - s.mag) ** 2
        f_dst = (
            -(c.beta_1 + c.beta_2 * s.mag) * np.log(s.dist_rup + 10)
            + c.gamma * s.dist_rup
        )
        f_ste = c.epsilon * np.log(s.v_s30)
        f_mec = c.phi * flag_mech

        ln_resp = f_mag + f_dst + f_ste + f_mec

        return ln_resp

    def _calc_ln_std(self) -> np.ndarray:
        s = self._scenario
        ln_std = (
            1.18
            + 0.035 * np.log(np.clip(self.PERIODS, 0.05, 3.0))
            - 0.06 * np.clip(s.mag, 5.0, 7.5)
        )
        return ln_std

In [19]:
from pygmm.model import Scenario

# Create an instance of the predictors
# We use values based on the original data file
scenario = Scenario(
    mag= 2.91732,
    dist_rup= 81.04359,
    v_s30=240.98754,
    mechanism="SS"
)

In [ ]:
model = Idriss2014(scenario)

In [21]:
# Calculate Metrics
ln_resp = model._calc_ln_resp()
ln_std = model._calc_ln_std()
# Print Accuracy
print(np.mean(ln_std))

0.8461553421379613
